In [1]:
import os
import getpass

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper

search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results=5)

In [3]:
from collections import defaultdict
from typing import List
import json

from langchain.output_parsers.openai_tools import (
    JsonOutputToolsParser,
    PydanticToolsParser,
)
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessage
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation

tool_executor = ToolExecutor([tavily_tool])
parser = JsonOutputToolsParser(return_id=True)

def execute_tools(state: List[BaseMessage]) -> List[BaseMessage]:
    tool_invocation: AIMessage = state[-1]
    parsed_tool_calls = parser.invoke(tool_invocation)
    ids = []
    tool_invocations = []
    for parsed_call in parsed_tool_calls:
        for query in parsed_call["args"]["search_queries"]:
            tool_invocations.append(
                ToolInvocation(
                    tool="tavily_search_results_json",
                    tool_input=query,
                )
            )
            ids.append(parsed_call["id"])
    outputs = tool_executor.batch(tool_invocations)
    print(f"outputs: {outputs}")
    outputs_map = defaultdict(dict)
    for id_, output, invocation in zip(ids, outputs, tool_invocations):
        outputs_map[id_][invocation.tool] = output
    print(f"outputs_map: {outputs_map}")
    result = [
        ToolMessage(content=json.dumps(query_outputs), tool_call_id=id_)
        for id_, query_outputs in outputs_map.items()
    ]
    return result

In [4]:
import datetime

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field, ValidationError
from langchain_openai import ChatOpenAI
from langsmith import traceable

actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a expert researcher.
             Current time: {time}
             1. {first_instruction}
             2. Reflect and critique your answer. Be severe to maximize improvement.
             3. Recommend search queries to research information and improve your answer.""",
        ),
        MessagesPlaceholder(variable_name="messages"),
        ("system", "Answer the user's question above using the required format."),
    ]
).partial(
    time=lambda: datetime.datetime.now().isoformat(),
)

class Reflection(BaseModel):
    missing: str = Field(description="Critique of what is missing.")
    superfluous: str = Field(description="Critique of what is superfluous.")

class AnswerQuestion(BaseModel):
    """Answer a question."""
    answer: str = Field(description="~250 word detailed answer to the question.")
    reflection: Reflection = Field(description="Your reflection on the initial answer.")
    search_queries: List[str] = Field(description="1-3 search queries for researching improvements to address the critique of your current answer.")

llm = ChatOpenAI(model="gpt-4-turbo-preview")
initial_answer_chain = actor_prompt_template.partial(
    first_instruction="Provide a detailed ~250 word answer."
) | llm.bind_tools(tools=[AnswerQuestion], tool_choice="AnswerQuestion")

validator = PydanticToolsParser(tools=[AnswerQuestion])

class ResponderWithRetries:
    def __init__(self, runnable, validator):
        self.runnable = runnable
        self.validator = validator

    @traceable
    def respond(self, state: List[BaseMessage]):
        print(f"state: {state}")
        response = []
        for attempt in range(3):
            try:
                response = self.runnable.invoke({"messages":state})
                self.validator.invoke(response)
                return response
            except ValidationError as e:
                state = state + [HumanMessage(content=repr(e))]
        return response

In [5]:
first_responder = ResponderWithRetries(
    runnable=initial_answer_chain,
    validator=validator,
)

In [6]:
example_question = "Why is reflection useful in AI?"
initial = first_responder.respond([HumanMessage(content=example_question)])

state: [HumanMessage(content='Why is reflection useful in AI?')]


In [7]:
initial

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qvdEMY1U4wqADOElENmEKj56', 'function': {'arguments': '{"answer":"Reflection in AI refers to the ability of artificial intelligence systems to assess and adapt their own processes and decision-making strategies. This introspective capability is crucial for several reasons. Firstly, it enables AI systems to learn from their experiences, improving their performance over time without direct human intervention. By analyzing their actions and outcomes, AI can identify patterns of success and failure, adjusting their algorithms accordingly to enhance future outcomes.\\n\\nSecondly, reflection in AI contributes to more transparent and explainable decision-making processes. As AI systems become more complex, understanding the rationale behind their decisions becomes increasingly important, especially in critical applications like healthcare, finance, and autonomous vehicles. Reflective AI can provide insights into its reasonin

In [8]:
parsed = parser.invoke(initial)
parsed

[{'type': 'AnswerQuestion',
  'args': {'answer': "Reflection in AI refers to the ability of artificial intelligence systems to assess and adapt their own processes and decision-making strategies. This introspective capability is crucial for several reasons. Firstly, it enables AI systems to learn from their experiences, improving their performance over time without direct human intervention. By analyzing their actions and outcomes, AI can identify patterns of success and failure, adjusting their algorithms accordingly to enhance future outcomes.\n\nSecondly, reflection in AI contributes to more transparent and explainable decision-making processes. As AI systems become more complex, understanding the rationale behind their decisions becomes increasingly important, especially in critical applications like healthcare, finance, and autonomous vehicles. Reflective AI can provide insights into its reasoning, making it easier for humans to trust and verify the AI's decisions.\n\nLastly, refl

In [7]:
revise_instructions = """Revise your previous answer using the new information.
    - You should use the previous critique to add important information to your answer.
        - You MUST include numerical citations in your revised answer to ensure it can be verified.
        - Add a "References" section to the bottom of your answer (which does not count towards the word limit). In form of:
            - [1] https://example.com
            - [2] https://example.com
    - You should use the previous critique to remove superfluous information from your answer and make SURE it is not more than 250 words.
"""

class ReviseAnswer(AnswerQuestion):
    """Revise your original answer to your question."""

    references: List[str] = Field(description="Citations motivating your updated answer.")

revision_chain = actor_prompt_template.partial(
    first_instruction=revise_instructions
) | llm.bind_tools(tools=[ReviseAnswer], tool_choice="ReviseAnswer")

revision_validator = PydanticToolsParser(tools=[ReviseAnswer])

revisor = ResponderWithRetries(
    runnable=revision_chain,
    validator=revision_validator,
)

In [20]:
revised = revisor.respond(
    [
        HumanMessage(content=example_question),
        initial,
        ToolMessage(
            tool_call_id=initial.additional_kwargs["tool_calls"][0]["id"],
            content=json.dumps(
                tavily_tool.invoke(str(parsed[0]["args"]["search_queries"]))
            ),
        ),
    ]
)

state: [HumanMessage(content='Why is reflection useful in AI?'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qvdEMY1U4wqADOElENmEKj56', 'function': {'arguments': '{"answer":"Reflection in AI refers to the ability of artificial intelligence systems to assess and adapt their own processes and decision-making strategies. This introspective capability is crucial for several reasons. Firstly, it enables AI systems to learn from their experiences, improving their performance over time without direct human intervention. By analyzing their actions and outcomes, AI can identify patterns of success and failure, adjusting their algorithms accordingly to enhance future outcomes.\\n\\nSecondly, reflection in AI contributes to more transparent and explainable decision-making processes. As AI systems become more complex, understanding the rationale behind their decisions becomes increasingly important, especially in critical applications like healthcare, finance, and autonomo

In [17]:
parsed = parser.invoke(revised)
parsed

[{'type': 'ReviseAnswer',
  'args': {'answer': 'Reflection in AI, the ability of systems to assess and adapt their own processes, is vital for several reasons. First, it enables AI to learn from experiences, improving performance over time without direct human intervention. This is crucial for developing systems that can adjust their algorithms based on outcomes, enhancing efficiency and effectiveness [1].\n\nSecond, reflective AI contributes to transparent and explainable decision-making. As AI systems grow more complex, it becomes increasingly important to understand the rationale behind their decisions, particularly in sensitive sectors like healthcare and autonomous driving [2]. Reflective AI can provide insights into its reasoning, fostering trust and allowing for human oversight.\n\nLastly, reflection allows AI to identify and mitigate biases in decision-making. AI systems often inherit biases from training data, which can lead to unfair outcomes. Through reflection, AI can recog

In [8]:
from langgraph.graph import END, MessageGraph

MAX_ITERATIONS = 5
builder = MessageGraph()
builder.add_node("draft", first_responder.respond)
builder.add_node("execute_tools", execute_tools)
builder.add_node("revise", revisor.respond)
# draft -> execute_tools
builder.add_edge("draft", "execute_tools")
# execute_tools -> revise
builder.add_edge("execute_tools", "revise")

# Define looping logic:


def _get_num_iterations(state: List[BaseMessage]):
    i = 0
    for m in state[::-1]:
        if not isinstance(m, (ToolMessage, AIMessage)):
            break
        i += 1
    return i


def event_loop(state: List[BaseMessage]) -> str:
    # in our case, we'll just stop after N plans
    num_iterations = _get_num_iterations(state)
    if num_iterations > MAX_ITERATIONS:
        return END
    return "execute_tools"


# revise -> execute_tools OR end
builder.add_conditional_edges("revise", event_loop)
builder.set_entry_point("draft")
graph = builder.compile()

In [9]:
events = graph.stream(
    [HumanMessage(content="How should we handle the climate crisis?")]
)
for i, step in enumerate(events):
    node, output = next(iter(step.items()))
    print(f"## {i+1}. {node}")
    print(str(output))
    print("---")

state: [HumanMessage(content='How should we handle the climate crisis?')]
## 1. draft
content='' additional_kwargs={'tool_calls': [{'id': 'call_TXNtXkeW8JIz0uf187hw3w7J', 'function': {'arguments': '{"answer":"Addressing the climate crisis requires a multi-faceted approach involving governments, businesses, communities, and individuals. Key strategies include transitioning to renewable energy sources, improving energy efficiency, and adopting sustainable agriculture and forestry practices. \\n\\nFirstly, transitioning to renewable energy sources such as solar, wind, and hydroelectric power is crucial to reduce greenhouse gas emissions from fossil fuels. Governments should incentivize renewable energy production and make significant investments in clean energy infrastructure. \\n\\nSecondly, improving energy efficiency in buildings, transportation, and industries can significantly reduce energy consumption and emissions. This can be achieved through stricter building codes, promoting ene

In [ ]:
print(parser.invoke(step[END][-1])[0]["args"]["answer"])